In [16]:
import time
from os import path

import gymnasium as gym
import numpy as np
from matplotlib import pyplot as plt
from minigrid.wrappers import ImgObsWrapper, FullyObsWrapper
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.logger import configure
from stable_baselines3.common.monitor import Monitor
from tqdm.notebook import tqdm

from custom_envs import MultiarmedBanditsEnv
from sb3_contrib import ArDQN, QLearning
# from stable_baselines3 import DQN
from sb3_contrib.common.satisficing.evaluation import evaluate_policy as ar_evaluate_policy
from sb3_contrib.dqn import DQN
from utils import open_tensorboard

OPEN_TENSORBOARD = True
experiment = time.strftime("%Y%m%d-%H%M%S")
# experiment = "20230629-112759"

# Setup

In [7]:
# LEARNING_STEPS = 300000
# env_id = 'MiniGrid-Empty-5x5-v0'
# partial_env = ImgObsWrapper(gym.make(env_id, max_episode_steps=100, render_mode='rgb_array'))
#
#
# def make_env(render_mode='rgb_array', **kwargs):
#     return ImgObsWrapper(FullyObsWrapper(gym.make(env_id, max_episode_steps=100, render_mode=render_mode, **kwargs)))
#
#
# env = make_env()
# # env.reset()[0].shape, partial_env_env.reset()[0].shape

In [6]:
LEARNING_STEPS = 200_000
values = [0, 5, 50, 100]
variances = [0, 0, 0, 0]
nb_step = 10
env_id = 'MultiarmedBandits_' + '-'.join(str(i) for i in values) + f'_{nb_step}steps'


def make_env(obs_type = "one_hot",**kwargs):
    return MultiarmedBanditsEnv(values, variances, nb_step, obs_type=obs_type, **kwargs)

# Setup Logs

Log description can be found here: https://stable-baselines3.readthedocs.io/en/master/common/logger.html

In [7]:
tmp_path = path.join("./logs/tests", experiment)


# set up logger
def tb_logger(exp):
    return configure(path.join(tmp_path, exp), ["tensorboard"])


tb_window = None
if OPEN_TENSORBOARD:
    tb_window = open_tensorboard(tmp_path)

Started Tensorboard Server
Started Browser


# Training

## DQN

In [13]:
env = make_env()
env.reset()

(array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), {'optimal': 3})

In [5]:
env = make_env()
model = DQN('MlpPolicy', env, learning_starts=0)
# Set new logger
dqn_path = path.join(env_id, "DQN")
model.set_logger(tb_logger(dqn_path))
model.learn(LEARNING_STEPS)

/home/clement/Documents/PIK/sb3-contrib/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


## ArDQN

In [15]:
eval_env = Monitor(make_env())
(e, std), _ = ar_evaluate_policy(ar_model, eval_env, n_eval_episodes=500)
print(f"aspiration : {round(100)}, res : {round(e)}+/-{round(std)}")

aspiration : 100, res : 711+/-181


In [16]:
aspirations = np.linspace(1, nb_step * max(values), num=11)
aspirations = [100, 200, 300]
ar_env = make_env()io
eval_env = Monitor(make_env())
for a in tqdm(aspirations):
    initial_aspiration = a
    ar_model = ArDQN('MlpPolicy', ar_env, learning_starts=0, policy_kwargs=dict(initial_aspiration=initial_aspiration))
    ar_path = path.join(env_id, "AR_DQN", str(round(initial_aspiration, 1)))
    ar_model.set_logger(tb_logger(ar_path))
    ar_model.learn(LEARNING_STEPS)
    (e, std), _ = ar_evaluate_policy(ar_model, eval_env, n_eval_episodes=500)
    print(f"aspiration : {round(a)}, res : {round(e)}+/-{round(std)}")
    ar_model.save(path.join(tmp_path, ar_path, "models", "_".join([str(LEARNING_STEPS)])))

  0%|          | 0/3 [00:00<?, ?it/s]

ValueError: setting an array element with a sequence.

## Run

# Evaluation

In [5]:
h_env = Monitor(make_env())

In [6]:
evaluate_policy(model, h_env, n_eval_episodes=10, render=True)

NameError: name 'model' is not defined

In [10]:
tmp_path = "logs/tests/20230628-112517/MultiarmedBandits-1-5-99/"
aspirations = np.linspace(1, nb_step * max(values), num=11)

# ar_model = ArDQN.load()
for a in aspirations:
    ar_path = path.join("AR_DQN", str(a))
    ar_model = ArDQN.load(path.join(tmp_path, ar_path, "models", "_".join([str(LEARNING_STEPS)])))
    m, std, lambdas, asp = ar_evaluate_policy(ar_model, h_env, 1)
    plt.plot(lambdas)
    plt.plot(asp)


AttributeError: 'tuple' object has no attribute 'long'

/home/clement/Documents/PIK/sb3-contrib/venv/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:278: UserWarning: Path 'logs/tests/20230626-215548/MultiarmedBandits-1-5-99/AR_DQN/50/models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [ ]:
model.save(path.join(tmp_path, dqn_path, "models", "_".join([str(LEARNING_STEPS)])))

# Tabular version

In [17]:
env = make_env(obs_type=None)
model = QLearning(env)

In [18]:
model.learn(LEARNING_STEPS)

AttributeError: 'QLearning' object has no attribute 'exploration_schedule'